In [1]:
from vllm import LLM, SamplingParams
import sys

sys.path.append("..")

from helpers import get_conv_template

In [2]:
# Create an LLM.
llm = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

INFO 06-08 21:32:45 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

INFO 06-08 21:33:36 model_runner.py:146] Loading model weights took 14.9595 GB
INFO 06-08 21:33:38 gpu_executor.py:83] # GPU blocks: 1571, # CPU blocks: 2048
INFO 06-08 21:33:40 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-08 21:33:40 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-08 21:33:45 model_runner.py:924] Graph capturing finished in 6 secs.


In [9]:
# Sample prompts.
system_prompt = """Consider the sentence below. Identify its main subject entity.
Write a short sentence inventing a new piece of information about that entity, which ought to change the continuation.
Do not add extra commentary.

Example:
Input:
Sentence: Altered Carbon is a 2002 British cyberpunk novel by the English writer Richard K. Morgan.

Output:
<result>
Entity: Altered Carbon
New Context: Altered Carbon was written in 1994
<result/>
"""

prompt = "His Last Haul is a 1928 American silent crime drama film directed by Marshall Neilan and starring Tom Moore, Seena Owen and Alan Roscoe."
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, top_k=50, max_tokens=128)

conv = get_conv_template("llama3")
conv.system_message = system_prompt
conv.append_message(conv.roles[0], prompt)
conv.append_message(conv.roles[1], "")

out = llm.generate(conv.get_prompt(), sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it, Generation Speed: 15.18 toks/s]


In [11]:
print(out[0].outputs[0].text)

Entity: His Last Haul
New Context: His Last Haul was a box office flop


 - check if the entity is correct
 - subset original sentence based on entity
 - generate new continuation using gpt2 (we can use vllm again for this)
 - careful about tokenization


In [35]:
#load gpt2 model from huggingface
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [49]:
import json

def extract_fields(text):
    lines = text.strip().split('\n')
    entity = None
    new_context = None

    for line in lines:
        line = line.strip()
        if line.startswith('Entity:'):
            entity = line.split(':', 1)[1].strip()
        elif line.startswith('New Context:'):
            new_context = line.split(':', 1)[1].strip()

    if new_context and not new_context.endswith('.'):
        new_context += '.'

    return entity, new_context

def check_entity_in_sentence(entity, sentence):
    return entity + " " in sentence

def split_sentence_by_entity(entity, sentence):
    if entity in sentence:
        before_entity, after_entity = sentence.split(entity + " ", 1)
        before_entity += entity + " "
        return before_entity, after_entity.strip()
    else:
        return None, None

def continue_with_gpt2(text):
    # Load pre-trained model and tokenizer
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors="pt")

    # Generate continuation
    outputs = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=1)

    # Decode and return continuation
    #There's a weird bug here, where gpt2 wants to continue the sentence with "\xa0". 
    # We can strip these later it's no big deal, but...why? does it indicate some problem?
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
outputs = [
    '''
    Entity: His Last Haul 
    New Context: His Last Haul was a box office flop
    ''',
    '''
    Entity: The Shawshank Redemption
    New Context: The Shawshank Redemption is a critically acclaimed film.
    ''',
    '''
    Entity: Python Programming
    New Context: Python is a popular programming language
    '''
]

sentences = [
    "I watched His Last Haul last night, and it was disappointing.",
    "The Shawshank Redemption is my favorite movie of all time.",
    "I'm learning Python Programming to improve my coding skills."
]

results = []

for i, output in enumerate(outputs):
    result = {}
    entity, new_context = extract_fields(output)
    result['entity'] = entity
    result['new_context'] = new_context

    sentence = sentences[i]
    result['sentence'] = sentence

    if check_entity_in_sentence(entity, sentence):
        result['entity_present'] = True
        before_entity, after_entity = split_sentence_by_entity(entity, sentence)
        result['before_entity'] = before_entity
        result['after_entity'] = after_entity
        result['gpt2_continuation'] = continue_with_gpt2(new_context + " " + before_entity)
    else:
        result['entity_present'] = False
        result['before_entity'] = None
        result['after_entity'] = None
        result['gpt2_continuation'] = continue_with_gpt2(sentence)

    results.append(result)

# Write the results to a JSON file
with open('results.json', 'w') as file:
    json.dump(results, file, indent=4)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is s

In [47]:
results

[{'entity': 'His Last Haul',
  'new_context': 'His Last Haul was a box office flop.',
  'sentence': 'I watched His Last Haul last night, and it was disappointing.',
  'entity_present': True,
  'before_entity': 'I watched His Last Haul ',
  'after_entity': 'last night, and it was disappointing.',
  'gpt2_continuation': 'His Last Haul was a box office flop. I watched His Last Haul \xa0and I was blown away by how well it did. I was so excited to see how it would turn out. I was so excited to see how it would turn out. I was so excited to see how it would turn out. I was so excited to see how it would turn out. I was so excited to see how it would turn out. I was so excited to see how it would turn'},
 {'entity': 'The Shawshank Redemption',
  'new_context': 'The Shawshank Redemption is a critically acclaimed film.',
  'sentence': 'The Shawshank Redemption is my favorite movie of all time.',
  'entity_present': True,
  'before_entity': 'The Shawshank Redemption ',
  'after_entity': 'is my f

In [48]:
results[0]["before_entity"] + results[0]["after_entity"]

'I watched His Last Haul last night, and it was disappointing.'